In [1]:
from selenium import webdriver
from msedge.selenium_tools import options as EdgeOptions
from msedge.selenium_tools import Edge, EdgeOptions
import time 

options = EdgeOptions()
options.add_argument("--lang=vi")

driver_path = 'C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe'

# Create a Microsoft Edge WebDriver instance with options
driver = Edge(executable_path=driver_path, options=options)

driver.get("https://www.google.com/maps/place/B%E1%BB%87nh+vi%E1%BB%87n+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Y+D%C6%B0%E1%BB%A3c+TP.HCM/@10.7552452,106.6618958,17z/data=!3m1!4b1!4m8!3m7!1s0x31752efa87dc6d6d:0x4feb0a1e755b6d62!8m2!3d10.7552399!4d106.6644707!9m1!1b1!16s%2Fg%2F11g6mdmpzh?entry=ttu")


C:\Users\NguyenDucHuy\AppData\Local\Temp\ipykernel_8148\2446717878.py:12: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path=driver_path, options=options)


In [2]:
print(driver.title)

University Medical Center - Google Maps


In [3]:
total_number_of_reviews = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
print(total_number_of_reviews)
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

1,774


In [4]:
import time

SCROLL_PAUSE_TIME = 10

scrollable_div = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]')

# Get scroll height
last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    # Scroll down to bottom
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

while True:
    try:
        see_more_buttons = driver.find_elements_by_css_selector('[aria-label="See more"]')
        if not see_more_buttons:
            break
        for button in see_more_buttons:
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[aria-label="See more"]'))).click()
            time.sleep(2)
    except Exception as e:
        print(e)
        break

In [ ]:
from bs4 import BeautifulSoup

def get_review_summary(result_set):
    rev_dict = {
        'Review Time': [],
        'Review Text' : [],
        'Review Rate': [],}
    for result in result_set:

        review_rate = result.find('span', class_='kvMYJc')["aria-label"]
        review_time = result.find('span', class_="rsqaWe").text.strip()
        review_text = result.find('span',class_='wiI7pd')
    
        if review_text is None:
            rev_dict['Review Text'].append('')
        else:
            rev_dict['Review Text'].append(review_text.text.strip())
        
        rev_dict["Review Time"].append(review_time)
        
        rev_dict['Review Rate'].append(review_rate)
    return rev_dict

In [ ]:
import pandas as pd
response = BeautifulSoup(driver.page_source, 'html.parser')
reviews = response.find_all('div', class_='jftiEf')
rev_dict = get_review_summary(reviews)

In [ ]:
print(len(rev_dict["Review Text"]))

In [ ]:
df = pd.DataFrame.from_dict(rev_dict)
df.columns = ["Time", "Review Text", "Rating"]

In [ ]:
df.to_csv("DHYD-review.csv",encoding='utf-8-sig')

In [ ]:
pd.read_csv("DHYD-review.csv", index_col="Unnamed: 0")